In [13]:
import torch
from torch import nn
import torch.nn.functional as F
import collections
import copy

# Encoder

edges

In [36]:
n_edges = 5
edge_features = torch.tensor([1.0, 2.0, 1.0, 1.0, 2.0])[:, None] # relative positions features
adjacency_list = [[0, 3], [1, 2], [2, 3], [2, 4], [3, 4]]

nodes

In [37]:
n_nodes = 5
# nodes are counted from 0
# features desciption
node_types = torch.tensor([1, 1, 0, 0, 0]) # 0 - interior, 1 - boundary
node_fields = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])[:, None] # field values
node_features = node_fields

dimensions

In [38]:
node_dim = 1
edge_dim = 1
(node_dim, edge_dim)

(1, 1)

collections

In [39]:
NodeSet = collections.namedtuple('NodeSet', ['N', 'node_types', 'features'])
EdgeSet = collections.namedtuple('EdgeSet', ['N', 'adjacency_list', 'features'])
MultiGraph = collections.namedtuple('Graph', ['node_set', 'edge_set'])

In [40]:
node_set = NodeSet(n_nodes, node_types, node_features)
edge_set = EdgeSet(n_edges, adjacency_list, edge_features)
graph = MultiGraph(node_set, edge_set)

# Processor

In [41]:
class StaticProcessor():
    def __init__(self, graph):
        self.graph = copy.deepcopy(graph)
        
        self.node_net = nn.Sequential(nn.Linear(node_dim + edge_dim, 10),
                                      nn.SiLU(),
                                      nn.Linear(10, node_dim))
    
    def update_nodes(self):
        edge_set = self.graph.edge_set
        node_set = self.graph.node_set
        
        for i in range(node_set.N):
            
            
            
            agg_edges = torch.zeros(edge_dim)
            
            for j in range(edge_set.N):
                if i in edge_set.adjacency_list[j]:
                    agg_edges += edge_set.features[j]
            
            X = torch.cat([node_set.features[i], agg_edges])
            
            node_set.features[i] = self.node_net(X)
    
    def forward(self):
        self.update_nodes()
        
        return self.graph